# SageWall: XGBoost Training Pipeline

**Author:** Andrew Zheng | **Date:** December 2025

---

## Read the DevLog!

Please visit the [DevLog](devlog.md) and read my fun and clear explanations for my design choices!

## Overview

This notebook trains and deploys a **binary classification model** to detect network intrusions in real-time. We use **XGBoost** (eXtreme Gradient Boosting) on AWS SageMaker to analyze network packet features and classify traffic as either **Normal** or **Attack**.

### Contents

| Section | Topic |
|---------|-------|
| **1. Setup** | Initialize AWS SDKs and configure IAM permissions |
| **2. Data Preparation** | Split dataset into training (80%) and validation (20%) |
| **3. Model Training** | Configure XGBoost hyperparameters and train on SageMaker |
| **4. Deployment** | Provision a real-time inference endpoint |
| **5. Testing** | Validate model accuracy on held-out samples |
| **6. Cleanup** | Delete resources to stop billing |

### Prerequisites

- AWS credentials configured (`aws configure`)
- Processed data in S3 bucket (`sagewall-processed-zheng-1b`)
- SageMaker execution role with appropriate permissions

---


## 1. Environment Setup & Data Preparation

### 1.1 Initializing AWS SDKs

We begin by importing the necessary libraries and establishing connections to AWS services:

| Library | Purpose |
|---------|---------|
| `sagemaker` | High-level SDK for ML training and deployment |
| `boto3` | Low-level AWS SDK for S3 operations |
| `pandas` | Data manipulation and CSV handling |

**IAM Role:** SageMaker requires an execution role with permissions to:
- Read/write to S3 buckets
- Create training jobs and endpoints
- Write logs to CloudWatch

The `get_execution_role()` function automatically retrieves this role when running in SageMaker Studio.

### 1.2 The 80/20 Split Strategy

Machine learning models need two datasets:

1. **Training Set (80%)**: The model learns patterns from this data
2. **Validation Set (20%)**: We test accuracy on data the model has *never seen*

> **Why split?** If we test on training data, we get falsely high accuracy. The model could simply "memorize" answers instead of learning patterns.

We use `random_state=42` to ensure reproducibility.

### 1.3 XGBoost Configuration

**Why XGBoost?** It's consistently the top-performing algorithm on tabular data competitions. Key hyperparameters:

| Parameter | Value | Explanation |
|-----------|-------|-------------|
| `max_depth` | 5 | Tree depth prevents overfitting (too complex = memorization) |
| `eta` | 0.2 | Learning rate smaller = more cautious learning |
| `gamma` | 4 | Minimum loss reduction regularization to prevent overfitting |
| `min_child_weight` | 6 | Minimum samples per leaf avoids learning from noise |
| `subsample` | 0.8 | Use 80% of data per tree reduces variance |
| `objective` | `binary:logistic` | Binary classification (Attack vs Normal) |
| `num_round` | 50 | Number of boosting rounds (trees to build) |

---


In [ ]:
import sagemaker
import boto3
import pandas as pd
import os
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput

# 1. Setup Environment
# -------------------
session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = session.boto_region_name
bucket_processed = 'sagewall-processed-zheng-1b'  # Your processed bucket
prefix = 'model-input'

print(f"Region: {region}")
print(f"Role: {role}")

# 2. Prepare Data (Split into Train/Validation)
# ---------------------------------------------
print("\nDownloading data from S3...")
s3 = boto3.client('s3')
# Download the clean file we made in Phase 2
s3.download_file(bucket_processed, 'KDDTrain+.txt', 'clean_data.csv')

# Read with Pandas (No headers, as per our Lambda format)
df = pd.read_csv('clean_data.csv', header=None)

# Split: 80% for Training, 20% for Validation (The "Exam")
train_data = df.sample(frac=0.8, random_state=42)
val_data = df.drop(train_data.index)

# Save locally without headers (SageMaker requirement)
train_data.to_csv('train.csv', index=False, header=False)
val_data.to_csv('validation.csv', index=False, header=False)

# Upload the split files back to S3
print("Uploading split datasets to S3...")
session.upload_data('train.csv', bucket=bucket_processed, key_prefix=prefix)
session.upload_data('validation.csv', bucket=bucket_processed, key_prefix=prefix)

s3_train_input = TrainingInput(s3_data=f's3://{bucket_processed}/{prefix}/train.csv', content_type='csv')
s3_val_input = TrainingInput(s3_data=f's3://{bucket_processed}/{prefix}/validation.csv', content_type='csv')
print("Data preparation complete.")

# 3. Define the XGBoost Model
# ---------------------------
# We retrieve the official XGBoost Docker image from AWS
container = image_uris.retrieve("xgboost", region, "1.5-1")

# Create the Estimator (The "Teacher")
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',  # Standard training instance
    output_path=f's3://{bucket_processed}/model-output/',
    sagemaker_session=session
)

# Set Hyperparameters (The "Teaching Style")
xgb_estimator.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective='binary:logistic',  # "Is it an Attack? Yes/No"
    num_round=50
)

# 4. Start Training
# -----------------
print("\nStarting Training Job... (This takes ~3-5 minutes)")
xgb_estimator.fit({'train': s3_train_input, 'validation': s3_val_input})

## 2. Model Deployment

### Creating a Real-Time Inference Endpoint

After training completes, the model artifact (a serialized XGBoost file) is saved to S3. To use it for predictions, we must **deploy** it to a live endpoint.

**What happens during deployment:**

```
┌─────────────────┐      ┌─────────────────┐      ┌─────────────────┐
│  Model Artifact │  →   │  Docker         │  →   │  EC2 Instance   │
│  (S3 .tar.gz)   │      │  Container      │      │  (ml.t2.medium) │
└─────────────────┘      └─────────────────┘      └─────────────────┘
```

1. SageMaker downloads the model from S3
2. Loads it into a pre-built XGBoost Docker container
3. Provisions an EC2 instance to host the API
4. Exposes an HTTPS endpoint for real-time predictions

### Instance Selection

| Instance | vCPU | Memory | Cost/Hour | Use Case |
|----------|------|--------|-----------|----------|
| `ml.t2.medium` | 2 | 4 GB | ~$0.05 | Development/Testing ✓ |
| `ml.m5.large` | 2 | 8 GB | ~$0.12 | Light Production |
| `ml.c5.xlarge` | 4 | 8 GB | ~$0.20 | High Throughput |

We use `ml.t2.medium` for cost-effective testing. For production workloads with high request volume, scale up to `ml.m5` or `ml.c5` instances.

> **Deployment takes 5-8 minutes** — SageMaker must provision infrastructure and health-check the endpoint.

---


In [ ]:
# 5. Deploy the Model
# -------------------
print("Deploying model to a real-time endpoint... (This takes 5-8 minutes)")

# We create a "Predictor" (The Interface)
xgb_predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',  # Cost-effective instance for inference
    serializer=sagemaker.serializers.CSVSerializer() # We will send it CSV data
)

print(f"\nModel deployed! Endpoint name: {xgb_predictor.endpoint_name}")

## 3. Model Testing & Validation

### Evaluating on Held-Out Data

Now we test the deployed model on our **validation set** — data the model has never seen during training. This gives us an unbiased estimate of real-world performance.

**Testing Process:**

1. Load 5 samples from `validation.csv`
2. Separate **features** (columns 1+) from **labels** (column 0)
3. Send features to the endpoint as CSV
4. Compare predictions to ground truth labels

### Understanding the Output

The model returns a **probability score** between 0.0 and 1.0:

| Score Range | Interpretation | Action |
|-------------|----------------|--------|
| 0.0 - 0.3 | High confidence: Normal | ✅ Allow traffic |
| 0.3 - 0.7 | Uncertain | ⚠️ Flag for review |
| 0.7 - 1.0 | High confidence: Attack | 🚨 Block & alert |

### SNS Alerting Integration

When the model detects a high-confidence threat (score > 0.90), we trigger an **AWS SNS alert** to notify security teams via email, SMS, or Slack webhook.

---


In [ ]:
import numpy as np
from alerts import send_alert

# 6. Test the Endpoint (FIXED VERSION)
# ------------------------------------
print("Testing the endpoint with a sample attack...")

# Configure your SNS Topic ARN for alerts (replace with your actual ARN)
SNS_TOPIC_ARN = 'arn:aws:sns:us-east-1:015283627438:sagewall-alerts'

# Grab the first 5 rows from our validation set
test_data = pd.read_csv('validation.csv', header=None)

# Separate answers (Col 0) and questions (Col 1+)
test_labels = test_data.iloc[:5, 0] 
test_samples = test_data.iloc[:5, 1:]

# Ask the AI for predictions
payload = test_samples.to_csv(header=False, index=False)
response = xgb_predictor.predict(payload).decode('utf-8')

# --- DEBUG: Show me exactly what the AI said ---
print(f"\n[DEBUG] Raw API Response:\n{response.strip()}") 
# -----------------------------------------------

# FIX: Replace newlines with commas so NumPy can read it
predictions = np.fromstring(response.replace('\n', ','), sep=',')

print("\n--- 🛡️ SAGEWALL DETECTION LOG ---")
for i in range(5):
    truth = "ATTACK" if test_labels.iloc[i] == 1 else "Normal"
    pred_score = predictions[i]
    ai_guess = "ATTACK" if pred_score > 0.5 else "Normal"
    
    if truth == ai_guess:
        status = "✅ CAUGHT" if truth == "ATTACK" else "✅ CLEARED"
    else:
        status = "❌ MISSED"
    
    print(f"Packet #{i+1}: Real={truth} | AI Confidence={pred_score:.4f} -> {status}")
    
    # Send SNS alert for high-confidence threats (score > 0.90)
    send_alert(pred_score, SNS_TOPIC_ARN)

## 4. Resource Cleanup

### Delete the Endpoint to Stop Billing

SageMaker endpoints are **billed by the hour** whether you're using them or not. A `ml.t2.medium` instance costs approximately **$0.05/hour** (~$36/month if left running).

**Always run this cell when you're done testing!**

The `delete_endpoint()` method:
1. Terminates the EC2 instance hosting the model
2. Deletes the endpoint configuration
3. Stops all billing for this resource

> **Note:** The trained model artifact remains in S3 — you can redeploy anytime by running Cell 1 again (after retraining or loading the estimator).

---

## Summary

You've successfully:

- [x] Configured AWS SageMaker environment
- [x] Split data into training/validation sets
- [x] Trained an XGBoost binary classifier
- [x] Deployed a real-time inference endpoint
- [x] Tested predictions on held-out data
- [x] Cleaned up resources to avoid charges

**Next Steps:**
- Use the Streamlit app (`app.py`) for interactive testing
- Integrate with Lambda for automated batch processing
- Set up CloudWatch alarms for endpoint monitoring


In [ ]:
# 7. CLEANUP
# ----------
xgb_predictor.delete_endpoint()
print("Endpoint deleted successfully.")